In [29]:
import network
import numpy as np

In [60]:
sizes = [4,2,3]
net = network.Network(sizes)

def printValues(listVals):
   for i in range(len(listVals)):
        print(listVals[i])
        print() 

In [98]:
sizes = [4,2,3]
net = network.Network(sizes)

print("original weights: (list of " + str(type(net.weights[0])) + ")")
print(type(net.weights[0]))
printValues(net.weights)
print("shape of net.weights[0]: " + str(net.weights[0].shape))


# manually set weights for connections between layers 0 and 1
net.weights[0]=np.array([[0.1, 0.2, 0.3, 0.4], [1.1,1.2,1.3,1.4]])
# same for layers 1 and 2
net.weights[1]=np.array([[0.5, -0.5], [0.6, -1], [0.7, -2]])

# manually set biases as well
net.biases[0] = np.array([[1], [1]])
net.biases[1] = np.array([[2], [2], [2]])

print("\nnew weights:")
printValues(net.weights)

print("new biases: (list of " + str(type(net.biases[0])) + ")")
print(type(net.biases[0]))
printValues(net.biases)

original weights: (list of <class 'numpy.ndarray'>)
<class 'numpy.ndarray'>
[[0.47723733 0.47416187 0.13750699 0.34584685]
 [0.39184332 0.96266928 0.10221322 0.94164865]]

[[0.60291293 0.94244211]
 [0.99866926 0.93833237]
 [0.94256869 0.15791888]]

shape of net.weights[0]: (2, 4)

new weights:
[[0.1 0.2 0.3 0.4]
 [1.1 1.2 1.3 1.4]]

[[ 0.5 -0.5]
 [ 0.6 -1. ]
 [ 0.7 -2. ]]

new biases: (list of <class 'numpy.ndarray'>)
<class 'numpy.ndarray'>
[[1]
 [1]]

[[2]
 [2]
 [2]]



In [134]:
# manually calculate output for given input
cur = np.array([[1], [2], [3], [4]]) # input

print("expected output = " + str(net.feedforward(cur)))

for layer in range(net.num_layers-1):
    tmp = []
    for node in range(len(net.weights[layer])):
        w = net.weights[layer][node]
        b = net.biases[layer][node]
        x = np.dot(w, cur) + b
        x = network.sigmoid(x)
        tmp.append(x[0])
    cur = tmp
    print("new cur = " + str(cur))
                
#for b, w in zip(self.biases, self.weights):                                                                                                                      
#a = sigmoid(np.dot(w, a)+b) 

expected output = [[0.87984967]
 [0.8305048 ]
 [0.66539081]]
new cur = [0.9820137900379085, 0.9999991684719722]
new cur = [0.8798496653691561, 0.8305048006614295, 0.665390813927719]
